<a href="https://colab.research.google.com/github/manavgurnani21/data_augmentation_tld_research/blob/main/Copy_of_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adding Images to Drive Folder

In [ ]:
!pip install -q kaggle

from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mbornoe/lisa-traffic-light-dataset
!unzip lisa-traffic-light-dataset.zip

ERROR: Operation cancelled by user


KeyboardInterrupt: ignored

In [ ]:
import os

# getting current directory
os.getcwd()

all_image_paths = []

## Getting Day Sequence Paths

In [ ]:
# getting all paths for content layer
content = os.listdir('/content/')
content.sort()
content = content[:-3]
content.remove('.config')
content.remove('kaggle.json')
content.remove('lisa-traffic-light-dataset.zip')
content.remove('Annotations')
content.remove('dayTrain')
content.remove('nightTrain')
for folder in content:
  if folder == '.ipynb_checkpoints':
    content.remove('.ipynb_checkpoints')
print(content)

In [ ]:
for folder in content:
  print('/content/' + folder + '/' + folder + '/frames/')
  list = os.listdir('/content/' + folder + '/' + folder + '/frames/')
  for path in list:
    all_image_paths.append('/content/' + folder + '/' + folder + '/frames/' + path)

## Getting Clip Paths

In [ ]:
train_paths = ['/content/dayTrain/dayTrain/', '/content/nightTrain/nightTrain/']
for path in train_paths:
  list1 = os.listdir(path)
  if '.DS_Store' in list1:
    list1.remove('.DS_Store')
  for name in list1:
    list2 = os.listdir(path + name + '/frames/')
    for item in list2:
      all_image_paths.append(path + name + '/frames/' + item)

In [ ]:
!unzip '/content/drive/MyDrive/cropped_images_randomized/cropped_images_randomized.zip'

# Adding All Annotations

## Getting all Sequence Annotations

In [ ]:
import os

all_annotation_paths = []

In [ ]:
root_path = '/content/Annotations/Annotations'
main = os.listdir(root_path)
main.remove('dayTrain')
main.remove('nightTrain')

for folder in main:
  list1 = os.listdir(root_path + '/' + folder)
  list1[0] = folder + list1[0]
  os.rename(root_path + folder + '/frameAnnotationsBOX.csv', root_path + folder + '/' + list1[0])
  all_annotation_paths.append(root_path + folder + '/' + list1[0])

## Getting all Clip Annotations

In [ ]:
clipPaths = [root_path + 'dayTrain/', root_path + 'nightTrain/']

for folder in clipPaths:
  list2 = os.listdir(folder)
  for name in list2:
    list3 = os.listdir(folder + name)
    list3[0] = name + list3[0]
    print(folder + name + '/' + list3[0])
    os.rename(folder + name + '/frameAnnotationsBOX.csv', folder + name + '/' + list3[0])
    all_annotation_paths.append(folder + name + '/' + list3[0])

# Sorting All Lists

In [ ]:
import numpy as np
image_paths = np.asarray(all_image_paths)
sorted_image_paths = np.sort(image_paths)
print(sorted_image_paths)

annotation_paths = np.asarray(all_annotation_paths)
sorted_annotation_paths = np.sort(annotation_paths)
print(sorted_annotation_paths)

# Cropping the Images

In [ ]:
def findIndexofElement(value, array):
  for i in range(len(array)):
    if array[i][array[i].rfind('/'):] == value:
      return i
      break

In [ ]:
os.mkdir('/content/allCroppedImages/')
os.mkdir('/content/allCroppedImages/stop/')
os.mkdir('/content/allCroppedImages/warning/')
os.mkdir('/content/allCroppedImages/go/')
os.mkdir('/content/allCroppedImages/warningLeft/')
os.mkdir('/content/allCroppedImages/goLeft/')
os.mkdir('/content/allCroppedImages/stopLeft/')
os.mkdir('/content/allCroppedImages/goForward/')

In [ ]:
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.utils import load_img
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import os

def cropAllImages(path):
  df = pd.read_csv(path, sep=';')
  filenames = df['Filename']
  leftX = np.asarray(df['Upper left corner X'])
  rightX = np.asarray(df['Lower right corner X'])
  leftY = np.asarray(df['Upper left corner Y'])
  rightY = np.asarray(df['Lower right corner Y'])
  tag = np.asarray(df['Annotation tag'])

  image_saved_counter = 0

  # loc_index is the location of the image path in all sorted paths
  for i in range(len(filenames)):
    findIndexofElement(filenames[i][filenames[i].rfind('/'):], sorted_image_paths)
    img = img_to_array(load_img(sorted_image_paths[findIndexofElement(filenames[i][filenames[i].rfind('/'):], sorted_image_paths)]))
    crop_img = array_to_img(img[leftY[i]:rightY[i], leftX[i]:rightX[i]])
    # inputting them in folder
    crop_img.save('/content/allCroppedImages/' + tag[i] + filenames[i][filenames[i].rfind('/'):])
    image_saved_counter+=1
    if(image_saved_counter%1000==0):
      print(image_saved_counter)

# for path in all_annotation_paths:
#   cropAllImages(path)

In [ ]:
# from google.colab import files
# !zip -r '/content/"allCroppedImages.zip"' '/content/allCroppedImages'
# files.download('/content/allCroppedImages.zip')

# Randomly Assigning Files

- after putting into sub-folders
- for each subfolder:
  - put all names in a list
  - shuffle
  - get all three indices
  - put into train, test, val

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
!unzip '/content/drive/MyDrive/cropped_images_randomized/cropped_images_randomized.zip' -d '/content/'

In [ ]:
# import os

# os.mkdir('/content/train/')
# os.mkdir('/content/test/')
# os.mkdir('/content/val/')

# folderList = ['train', 'test', 'val']
# for name in folderList:
#   os.mkdir('/content/' + name + '/stop/')
#   os.mkdir('/content/' + name + '/go/')
#   os.mkdir('/content/' + name + '/warning/')
#   os.mkdir('/content/' + name + '/warningLeft/')
#   os.mkdir('/content/' + name + '/goLeft/')
#   os.mkdir('/content/' + name + '/stopLeft/')
#   os.mkdir('/content/' + name + '/goForward/')

In [ ]:
# import random
# import shutil
# import pandas as pd
# import numpy as np

# def shuffleSelection(path):
#   allFolders = listPaths(path)
#   for folder in allFolders:
#     df = pd.DataFrame(listPaths(folder))
#     trainPaths, testPaths, valPaths = np.split(df, [int(.8 * len(df)), int(.9 * len(df))])
#     moveToFolder(trainPaths, testPaths, valPaths)

# def moveToFolder(trainPaths, testPaths, valPaths):
#   finalTrainPathList = np.asarray(trainPaths[0])
#   type(finalTrainPathList)
#   finalTestPathList = np.asarray(testPaths[0])
#   finalValPathList = np.asarray(valPaths[0])
#   for path in finalTrainPathList:
#     shutil.move(path[:-1], '/content/train' + path[33:-1])
#   for path in finalTestPathList:
#     shutil.move(path[:-1], '/content/test' + path[33:-1])
#   for path in finalValPathList:
#     shutil.move(path[:-1], '/content/val' + path[33:-1])

In [ ]:
# shuffleSelection('/content/content/allCroppedImages/')

In [ ]:
# from google.colab import files
# !zip -r '/content/cropped_images_randomized.zip' '/content/cropped_images_randomized'
# files.download('/content/cropped_images_randomized.zip')

In [ ]:
# import os, shutil
# folder = '/Users/manavgurnani21/Downloads/content/all_images_randomized/val/warningLeft'
# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
import os, shutil

# setting paths
folder_raw = '/Users/manavgurnani21/Downloads/cropped_images_raw/'
folder_augmented = '/Users/manavgurnani21/Downloads/content/cropped_images_randomized/'
folder_final = '/Users/manavgurnani21/Downloads/content/all_images_randomized/'

# used to get paths of all folders we are going to move into the big folder
all_sub_paths = []

# all class names
all_class_names = ['go', 'stopLeft', 'warningLeft', 'goForward', 'stop', 'warning', 'goLeft']

# for file in os.listdir(folder_raw):
#     print(file)

# getting paths of all sub  folders to copy
for filename in os.listdir(folder_raw):
    if '.DS_Store' not in filename:
        file_path = os.path.join(folder_raw, filename)
        for classifier in os.listdir(file_path):
            if '.DS_Store' not in classifier:
                all_sub_paths.append(os.path.join(file_path, classifier))

for filename in os.listdir(folder_augmented):
    if '.DS_Store' not in filename:
        file_path = os.path.join(folder_augmented, filename)
        for classifier in os.listdir(file_path):
            if '.DS_Store' not in classifier:
                all_sub_paths.append(os.path.join(file_path, classifier))

# creating base test train val folders
# os.mkdir('/Users/manavgurnani21/Downloads/content/all_images_randomized/train')
# os.mkdir('/Users/manavgurnani21/Downloads/content/all_images_randomized/test')
# os.mkdir('/Users/manavgurnani21/Downloads/content/all_images_randomized/val')

# for folder in os.listdir(folder_final):
#     for name in all_class_names:
#         os.mkdir(folder_final + folder + '/' + name)

for item in all_sub_paths:
    for image in os.listdir(item):
        directories = item.split('/')
        src = item + '/' + image
        dst = folder_final + directories[-2] + '/' + directories[-1] + '/'
        shutil.copy2(src, dst)

# Test Labels

Downloading, unzipping, and receiving dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/cropped_images_randomized/cropped_images_randomized.zip'

# !unzip '/content/cropped_images_randomized.zip'

In [ ]:
# getting the paths for all images in any folder
import random

def listPaths(path):
  pathList = []
  for folder in os.listdir(path):
    if folder == '.ipynb_checkpoints':
      continue
    pathList.append(path + '/' + folder)
  return pathList

In [ ]:
# getting the class names for each image in order ()
import os
all_image_path_train_labels = []
all_image_path_test_labels = []

for folder in listPaths('/content/content/cropped_images_randomized/train'):
    if folder != '/content/content/cropped_images_randomized/train/.DS_Store':
        for file in os.listdir(folder):
          all_image_path_train_labels.append(folder.split('/')[-1])

for folder in listPaths('/content/content/cropped_images_randomized/test'):
    if folder != '/content/content/cropped_images_randomized/test/.DS_Store':
        for file in os.listdir(folder):
          all_image_path_test_labels.append(folder.split('/')[-1])

In [ ]:
# print(all_image_path_train_labels)

# Training Dataset

In [ ]:
# There are several ways you can enhance your image detection model:

# Use a more powerful model architecture: There are many different model architectures that can be used for image detection, such as VGG, ResNet, and Inception. Using a more powerful architecture can help your model learn more discriminative features and improve performance.

# Fine-tune a pre-trained model: Pre-trained models have already been trained on a large dataset and can be fine-tuned for your specific task. Fine-tuning can help your model learn task-specific features and improve performance.

# Use transfer learning: Transfer learning involves using the features learned by a pre-trained model and applying them to a new task. This can be an effective way to improve performance on a new task, particularly if you have a limited amount of training data.

# Experiment with different hyperparameters: Hyperparameters are settings that determine the model's behavior and performance. You can try tuning different hyperparameters, such as the learning rate, batch size, and optimization algorithm, to see if they have an impact on your model's performance.

In [ ]:
# # importing the modules
# import os
# import shutil

# # Providing the folder path
# origin = '/Users/manavgurnani21/Downloads/content 2/cropped_images_non_augmented/train/go/'
# target = '/Users/manavgurnani21/Downloads/content/cropped_images_randomized/train/go/'

# # Fetching the list of all the files
# files = os.listdir(origin)

# shutil.copy2(origin, '/Users/manavgurnani21/Downloads/GitHub/data_augmentation_tld_research/random')

# print(files)

In [ ]:
# # getting the frequencies for setting the initial class weights and the distribution of the images in each class

# import matplotlib.pyplot as plt

# # Define the data for each class
# go_freq = all_image_path_test_labels.count('go')
# goForward_freq = all_image_path_test_labels.count('goForward')
# goLeft_freq = all_image_path_test_labels.count('goLeft')
# stop_freq = all_image_path_test_labels.count('stop')
# stopLeft_freq = all_image_path_test_labels.count('stopLeft')
# warning_freq = all_image_path_test_labels.count('warning')
# warningLeft_freq = all_image_path_test_labels.count('warningLeft')

# total = len(all_image_path_test_labels)

# # Combine the data into a single list
# data = [go_freq/total, goForward_freq/total, goLeft_freq/total, stop_freq/total, stopLeft_freq/total, warning_freq/total, warningLeft_freq/total]

# # Define the labels for each class
# labels = ['Go', 'Go Forward', 'Go Left', 'Stop', 'Stop Left', 'Warning', 'Warning Left']

# # Create the histogram
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])

# # Plot the data
# ax.bar(labels, data)

# # Add title and labels
# ax.set_title('Class Frequency Histogram')
# ax.set_xlabel('Class')
# ax.set_ylabel('Frequency')

# # Show the graph
# plt.show()

In [ ]:
# making the train test and validation split from the 'content' directory

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/content/cropped_images_randomized/train'
test_data_dir = '/content/content/cropped_images_randomized/test'
val_data_dir = '/content/content/cropped_images_randomized/val'

img_height = 180
img_width = 180
batch_size= 32

# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   train_data_dir,
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Load the images from the directory and convert them to a numpy array
train_data = ImageDataGenerator().flow_from_directory(
    train_data_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical'
)

# Get the numpy array of images and labels
images, labels = train_data.next()

images = np.asarray(images)
labels = np.asarray(labels)

# Print the shape of the numpy arrays
print(type(images))  # should print (32, 180, 180, 3)
print(type(labels))  # should print (32, num_classes)

Found 5096 files belonging to 7 classes.
Found 5098 files belonging to 7 classes.
Found 40757 images belonging to 7 classes.
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## Oversampling

- rotating images between -5 and 5 degrees
- scaling images from 0.9 to 1.1

In [ ]:
import os, shutil
import torch
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

# print(len(os.listdir('/content/content/cropped_images_randomized/val')))

# for end_class in os.listdir('/content/content/cropped_images_randomized/val'):
#   print(end_class + ": " + str(len(os.listdir('/content/content/cropped_images_randomized/val/' + end_class))))

# def getMaxNumImages(path):
#   numImagesInClass = []
#   for end_class in os.listdir(path):
#     print(end_class + ": " + str(len(os.listdir(path + end_class))))
#     numImagesInClass.append(len(os.listdir(path + end_class)))
#   return max(numImagesInClass)

# getMaxNumImages('/content/content/cropped_images_randomized/val/')

img_height = 180
img_width = 180
img_depth = 3
batch_size= 32

X = images.reshape((-1, 180*180*3))
smote = SMOTE(sampling_strategy='minority')
# adasyn = ADASYN(sampling_strategy='minority')
oversample = RandomOverSampler(sampling_strategy='minority')
X_resampled, y_resampled = oversample.fit_resample(X, labels)

train_ds = tf.data.Dataset.from_tensor_slices((X_resampled, y_resampled))
train_ds = train_ds.map(lambda x, y: tf.reshape(x, [img_width, img_height, img_depth]))
train_ds = train_ds.shuffle(buffer_size=len(X_resampled)).batch(batch_size)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
# normalizing the images and the first layer to fit the images

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(train_ds))
# first_image = image_batch[0]
# first_label = labels_batch[0]
# Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))

In [ ]:
# Define the number of unique labels
num_classes = 7

# Define a function to one-hot encode the labels
def one_hot_encode(y):
    return tf.one_hot(y, depth=num_classes)

# Use the map method to apply the one-hot encoding function to each batch of labels
# encoded_ds = normalized_ds.map(lambda x, y: (x, one_hot_encode(y)))

In [ ]:
# import tensorflow as tf
# import numpy as np
# from keras import backend as K

# Compute the class weights
# weights = np.asarray([1/(go_freq/total), 1/(goForward_freq/total), 1/(goLeft_freq/total), 1/(stop_freq/total), 1/(stopLeft_freq/total), 1/(warning_freq/total), 1/(warningLeft_freq/total)])

# np_1 = np.asarray([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0])
# np_2 = np.asarray([0.1, 0.4, 0.1, 0.1, 0.2, 0.1, 0.1])

# Define the weighted cross-entropy loss function (DO NOT NEED BECAUSE WE ARE USING TF'S FUNCTION)
# def loss(y_true, y_pred):
#   # scale predictions so that the class probas of each sample sum to 1
#   y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
#   # clip to prevent NaN's and Inf's
#   y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
#   # calc
#   loss = y_true * K.log(y_pred) * weights
#   loss = -K.sum(loss, -1)
#   return loss

# loss(np_1, np_2)

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import datetime
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

In [ ]:
%load_ext tensorboard

# ResNet 50

Building neural network and training.

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',classes=7,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(7, activation='softmax'))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 24,640,391
Trainable params: 1,052,679
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        print(y_true)
        y_true = one_hot_encode(y_true)
        print(y_pred)
        print(weights)
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss_true = -K.sum(loss, -1)
        return loss_true

    return loss

# y_true = tf.convert_to_tensor(np.array([0, 1, 0, 0]))
# y_pred = tf.convert_to_tensor(np.array([-18.6, 0.51, 2.94, -12.8]))
# class_weights = [1, 1, 1, 1, 1, 1, 1]
# bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# print(bce(y_true, y_pred).numpy())
# print((weighted_categorical_crossentropy(y_true, y_pred, class_weights)))

In [ ]:
import numpy as np

# Step 1: compute the number of instances for each class
class_counts = [18626, 159, 1874, 18591, 10367, 1106, 228]
N = sum(class_counts)

# Step 2: compute the empirical distribution of each class
class_distribution = [count / N for count in class_counts]

# Step 3: compute the expected loss for each class
def loss_function(i, j, distribution):
    if i == j:
        return -np.log(distribution[j])
    else:
        return -np.log(1-distribution[j])

expected_loss = []
for i in range(len(class_counts)):
    loss = 0
    for j in range(len(class_counts)):
        loss += class_distribution[j] * loss_function(i, j, class_distribution)
    expected_loss.append(loss)

# Step 4: compute the weight for each class
class_weights = [1 / np.sqrt(l) for l in expected_loss]
print(class_weights)

[1.311168735143627, 1.584821667513439, 1.4138638939277697, 1.3103777448742893, 1.2329410854948, 1.469933476990528, 1.57264307030534]


In [ ]:
# setting the initial class weights and compiling the network
# class_weights = {'dense_1' : 1/(go_freq/total), 'dense_2' : 1/(goForward_freq/total), 'dense_3' : 1/(goLeft_freq/total), 'dense_4' : 1/(stop_freq/total), 'dense_5' : 1/(stopLeft_freq/total), 'dense_6' : 1/(warning_freq/total), 'warningLeft' : 1/(warningLeft_freq/total)}
# class_weights = tf.constant([0.01324651, 0.15182218, 0.01278938, 0.01327909, 0.02388292, 0.22484979, 0.55932912])
# class_weights = [1/(go_freq/total), 1/(goForward_freq/total), 1/(goLeft_freq/total), 1/(stop_freq/total), 1/(stopLeft_freq/total), 1/(warning_freq/total), 1/(warningLeft_freq/total)]
class_weights = [1, 1, 1, 1, 1, 1, 1]
# class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(all_image_path_train_labels), y=all_image_path_train_labels)
# class_weights = [1 / np.sqrt(l) for l in expected_loss]
print(class_weights)

resnet_model.compile(optimizer='Adam', loss = tf.losses.SparseCategoricalCrossentropy, metrics=['accuracy'])

[1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(class_weights)

[1, 1, 1, 1, 1, 1, 1]


In [ ]:
# training the neural network with early callback for prevention of over/undertraining (given room of 50 epochs)

from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
resnet_history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2,
    callbacks=[callback, tensorboard_callback]
  )

Epoch 1/2


ValueError: ignored

In [ ]:
# precision = resnet_model.evaluate(val_ds)[1]
# print(f"Precision after epoch {epoch+1}: {precision}")

## TensorBoard Analysis

In [ ]:
%tensorboard --logdir logs/fit

## Prediction

In [ ]:
print(len(resnet_history.history['loss']))

In [ ]:
# resnet_model = keras.models.load_model('/Users/manavgurnani21/Downloads/Trained_Models/Experiment_3')

In [ ]:
# running predictions through the test dataset

pred=resnet_history.model.predict(test_ds)
print(pred[0])

In [ ]:
# plotting the training progress of the model

fig1 = plt.gcf()
plt.plot(resnet_history.history['accuracy'])
plt.axis(ymin=0,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
# calculating the accuracy of the model by comparing the model's labels with the actual labels

import numpy as np
pred_np = np.array(pred)

correctPred = 0
pred_np = np.asarray(pred)
correct = np.concatenate([y for x, y in test_ds], axis=0)

i = 0
for prediction in pred_np:
  if(np.argmax(pred_np[i]) == correct[i]):
        correctPred += 1
  i += 1

print(correctPred)
accuracy = correctPred / len(all_image_path_test_labels)

print("The accuracy of this model is:", accuracy)

In [ ]:
# predicted_classes = np.argmax(pred, axis=1)

# cm = confusion_matrix(class_names, predicted_classes)

# print(cm)

In [ ]:
from keras.models import save_model

# keras.models.save_model(resnet_model,'/Users/manavgurnani21/Downloads/Experiment_0_1')

In [ ]:
# keras.models.load_model('/Users/manavgurnani21/Downloads/Experiment_0_1')

# Data Augmentation

In [ ]:
# import required module
import os
import cv2
import numpy as np

def applyTransformation(directory):
    # iterate over files in
    # that directory
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if(filename.__contains__('.DS_Store') == False):
                img2 = cv2.imread(os.path.join(root, filename))
                new = specification(os.path.join(root, filename))
                new_2 = histogram_equalization(new)
                new_3 = log_inverse(new_2)
                new_4 = gamma(new_3, 0.25)
                cv2.imwrite(os.path.join(root, filename), new_4)

applyTransformation('/Users/manavgurnani21/Downloads/content/cropped_images_randomized/train')

## Experiment 1: Hist. Equalization

In [ ]:
import cv2
import numpy as np

def histogram_equalization(img_in):
# segregate color streams
    b,g,r = cv2.split(img_in)
    h_b, bin_b = np.histogram(b.flatten(), 256, [0, 256])
    h_g, bin_g = np.histogram(g.flatten(), 256, [0, 256])
    h_r, bin_r = np.histogram(r.flatten(), 256, [0, 256])
# calculate cdf
    cdf_b = np.cumsum(h_b)
    cdf_g = np.cumsum(h_g)
    cdf_r = np.cumsum(h_r)

# mask all pixels with value=0 and replace it with mean of the pixel values
    cdf_m_b = np.ma.masked_equal(cdf_b,0)
    cdf_m_b = (cdf_m_b - cdf_m_b.min())*255/(cdf_m_b.max()-cdf_m_b.min())
    cdf_final_b = np.ma.filled(cdf_m_b,0).astype('uint8')

    cdf_m_g = np.ma.masked_equal(cdf_g,0)
    cdf_m_g = (cdf_m_g - cdf_m_g.min())*255/(cdf_m_g.max()-cdf_m_g.min())
    cdf_final_g = np.ma.filled(cdf_m_g,0).astype('uint8')
    cdf_m_r = np.ma.masked_equal(cdf_r,0)
    cdf_m_r = (cdf_m_r - cdf_m_r.min())*255/(cdf_m_r.max()-cdf_m_r.min())
    cdf_final_r = np.ma.filled(cdf_m_r,0).astype('uint8')
# merge the images in the three channels
    img_b = cdf_final_b[b]
    img_g = cdf_final_g[g]
    img_r = cdf_final_r[r]

    img_out = cv2.merge((img_b, img_g, img_r))
# validation
    equ_b = cv2.equalizeHist(b)
    equ_g = cv2.equalizeHist(g)
    equ_r = cv2.equalizeHist(r)
    equ = cv2.merge((equ_b, equ_g, equ_r))
    #print(equ)
    #cv2.imwrite('output_name.png', equ)
    return img_out

## Experiment 2: Logarithm and Inverse

In [ ]:
def log_inverse(image):
    c = 255 / np.log(1 + np.max(image))
    log_image = c * (np.log(image + 1))

    # Specify the data type so that
    # float value will be converted to int
    log_image = np.array(log_image, dtype = np.uint8)

    img = cv2.cvtColor(log_image, cv2.COLOR_BGR2RGB)
    colored_negative = abs(255-img)
    return colored_negative

## Experiment 3: Gamma

In [ ]:
def gamma(src, gamma):
    invGamma = 1 / gamma

    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)

    return cv2.LUT(src, table)

## Experiment 4: Specification

In [ ]:
import matplotlib.pyplot as plt
import cv2
from skimage import data
from skimage import exposure
from skimage.exposure import match_histograms
from PIL import Image

def specification(path):
    reference_unsized = cv2.cvtColor(cv2.imread('/Users/manavgurnani21/Downloads/content/cropped_images_randomized/Colour-Wheel-Rainbow-Spectrum-Color-Wheel-1740381.jpg'), cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    reference = cv2.resize(reference_unsized, (image.shape[1], image.shape[0]))
    matched = match_histograms(image, reference, multichannel=True)
    return matched

## Agenda for 10/18

- sort out issue with random shuffle function (ask about cropping time)
- find way to convert images to dataset
  - ask why we need singular class folders

Goals for the next two weeks:
- run experiments (and caputre results)
- finish research paper